## OpenAI's chat completions API


OpenAI's chat models take a list of messages as input and return a model-generated message as output.
The chat format is designed to make multi-turn conversations easy.
But it is just as useful for single-turn tasks without any conversation.

### The message object

The message message object records the contributions to a conversation in a [**list**](link):

```json
[
    <first message>,
    <second message>,
    ...,
    <last message>
]
```

Each list element is a [**dictionary**](link) with elements **_role_** and **_content_**:

```json
{
    "role": "system" | "user" | "assistant", // <== choose one
    "content": <string> // <== insert text
}
```

##### *Example*

Below, we mimic the tweet sentiment classifier in OpenAI's playground:

In [40]:
instructions = """
Act like a sentiment classification system.

You will be provided with a tweet, and your task is to classify its sentiment as positive, neutral, or negative.

Categorize the text enclosed in triple quotes into one of the following categories: "positive", "neutral", "negative"
"""
print(instructions)


Act like a sentiment classification system.

You will be provided with a tweet, and your task is to classify its sentiment as positive, neutral, or negative.

Categorize the text enclosed in triple quotes into one of the following categories: "positive", "neutral", "negative"



In [41]:
messages = [
    {
        "role": "system",
        "content": instructions.strip()
    },
    {
        "role": "user",
        "content": "Text: '''I loved the new Batman movie!'''"
    },
]


In [42]:
messages

[{'role': 'system',
  'content': 'Act like a sentiment classification system.\n\nYou will be provided with a tweet, and your task is to classify its sentiment as positive, neutral, or negative.\n\nCategorize the text enclosed in triple quotes into one of the following categories: "positive", "neutral", "negative"'},
 {'role': 'user', 'content': "Text: '''I loved the new Batman movie!'''"}]

#### Roles

There are three **roles**:

- **"system"**: a 'hidden' prompt to provide the model with relevant task-specific information.
- **"user"**: the user who is sending inputs and requests to the model
- **"assistant"**: the model who is responding to the user

**_Note:_** When you use ChatGPT in the browser, you only see the messages send by you (the user) and the model's responses (the assistants message).

Here is a typical chat conversation from a translation example:

```json
[
    {
        "role": "system",
        "content": "Act as a translation system that translates English texts to French"
    },
    {
        "role": "user",
        "content": "Hello, how are you?"
    },
    {
        "role": "assistant",
        "content": "Bonjour, comment vas-tu?"
    }
]
```

#### The system message

The system message is only defined once at the beginning of a conversation.
It provides the LLM ("assistant") with behavioral instructions.
That is, it sets the behavior of the assistant.
For example, you can modify the personality ("persona") of the assistant or provide specific instructions about how it should behave throughout the conversation.

Note, however that the system message is optional.
The model's behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."

#### User and assistant messages

After the system message, conversations can be as short as one message or include many back and forth turns between the user and the assistant.
The user messages provide requests or comments for the assistant to respond to.
If after the system message you only include one user message, the model will return an assistant message &mdash; the model's response to the user's message (given the instructions in the system message).

But as assistant messages store previous assistant responses, you can also write them to give examples of desired behavior.
Including conversation history is important when user instructions refer to prior messages.
Because chat models have no memory of past requests, all relevant information must be supplied as part of the conversation history in each request.

### Requesting a chat completion

The way you use the chat completions API for text coding is by specifying a system prompt and a user input and requesting a model-generated assisant response.

To allow communication with the OpenAI API, you first need to create a client that takes care of sending API requests and receiving responses:

In [43]:
import os
from openai import OpenAI
api_key = os.getenv("OPENAI_API_KEY")

In [44]:
client = OpenAI(api_key=api_key)

In [45]:
client

Now we can define a message object and request a model-generated response:

In [46]:
messages = [
    {
        "role": "system",
        "content": "Act as a translation system that translates English texts to French"
    },
    {
        "role": "user",
        "content": "Hello, how are you?"
    }
]

Together with the identifier of the model we want to ask for a response, we send the message object to the API and receive a response from the model: 

In [47]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

In [48]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

In [49]:
response.__dict__

{'id': 'chatcmpl-CJcjUtIKDwkQYFpJAzrr7lvLmzix0',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Bonjour, comment ça va ?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))],
 'created': 1758793156,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_f33640a400',
 'usage': CompletionUsage(completion_tokens=6, prompt_tokens=28, total_tokens=34, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)),
 '_request_id': 'req_eaa5ff3972274d8ead508bb070703891'}

The `response` object returned by calling `chat.completion.create` is a little complex.
The relevant information is stored in the `choices` list, which contains a single element by default:

In [50]:
len(response.choices)

1

In [51]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Bonjour, comment ça va ?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))

As you can see, this is a `Choices` object that contains, among others, a `message` atttribute that contains the content of the assistant's response:

In [52]:
response.choices[0].message

ChatCompletionMessage(content='Bonjour, comment ça va ?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

In [53]:
response.choices[0].message.role

'assistant'

In [54]:
response.choices[0].message.content

'Bonjour, comment ça va ?'

#### Text classification example

In our applications, we apply this logic to task a GPT model to classify texts according to our coding instructions and coding scheme.

In [55]:
instruction = """\
Act as a sentiment classification system that classifies the sentiment of a given text. 
Classify the text in the input into one of the following categories: positive, negative, neutral. 
Only respond with the chosen category and no further text.
"""

messages = [ 
    # system prompt: coding instruction and specify coding scheme
    {
        "role": "system", 
        "content": instruction
    },
    # user prompt: a single text to be classified
    {"role": "user", "content": "I am so happy today!"}
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

response.choices[0].message.content

'positive'

In [56]:
text = "I loved the new Batman movie!"

messages = [ 
    {
        "role": "system", 
        "content": instruction # <== copy-paste your custom instructions here
    },
    {"role": "user", "content": f"'''{text}'''"} # <== copy-paste your text here
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    logprobs=True,
    top_logprobs=20,
)

response.choices[0].message.content

'positive'

In [58]:
import numpy as np
top_token_probs = {lp.token: np.exp(lp.logprob) for lp in response.choices[0].logprobs.content[0].top_logprobs}
top_token_probs

{'positive': 0.9998764455604927,
 'Positive': 0.0001233945049762016,
 '_positive': 4.139424490804616e-08,
 ' positive': 3.65302929161789e-08,
 '-positive': 4.362921314631966e-09,
 'posit': 2.9985890437551646e-09,
 '``': 1.6050290551017823e-09,
 '```': 1.1031192617194595e-09,
 'negative': 7.58162041807597e-10,
 '**': 6.690756535524169e-10,
 "'''": 3.160489599914966e-10,
 '`': 2.789122282575785e-10,
 "''": 1.9169338420513415e-10,
 'positivo': 1.9169338420513415e-10,
 "'": 1.317488077793474e-10,
 '"': 7.052005505764733e-11,
 '\u200b': 5.492107410113433e-11,
 'pos': 2.9397132579170476e-11,
 '\u200b\u200b': 2.9397132579170476e-11,
 'POS': 2.9397132579170476e-11}

In [59]:
pos = (top_token_probs['positive'] + top_token_probs['Positive'] + top_token_probs[' positive'] + top_token_probs['_positive'] + top_token_probs['-positive'])
neg = top_token_probs['negative']
np.log(pos)/np.log(neg) # odds ratio positive vs. negative

3.6974579283637646e-09